This notebook is for peak matching and pivoting the df from long to wide format for ML

In [ ]:
import pandas as pd

df_main = pd.read_csv('Desktop/Jupyter Area/capstone-project/df_main1.csv')

# Checking for NAs and Nulls
print('CAS Null Count')
print(df_main['CAS'].isnull().sum())
print('CAS NA Count')
print(df_main['CAS'].isna().sum())
print('ID Null Count')
print(df_main['ID'].isnull().sum())
print('ID NA Count')
print(df_main['ID'].isna().sum())

# If there are, filter them out?

# Group peaks by CAS and RT values
    # same CAS might show up at different RT so they should be separate features
        #  Column bleed, isomer, co-elutions

# Create unique PeakID (CAS+RT)

# Keep representative RTs for reporting
    # sure, you'll know the CAS, but it will just say group1 or group2 etc and you'll want to know the RT of that group

# Pivot time